In [1]:
import click, random, os, pickle, time, sys
import google.oauth2.credentials
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from pathlib import Path
import json
import db
import isodate


In [2]:
SCOPES = ['https://www.googleapis.com/auth/youtube.readonly']
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'

# Gestion des chemins pour Windows
HOME = str(Path.home())
CACHE_CREDENTIALS_DIR = os.path.join(HOME, ".ytube")
CACHE_CREDENTIALS_FILE = os.path.join(CACHE_CREDENTIALS_DIR, "token.pickle")

# Permet les connexions non sécurisées pour le développement local
os.environ['OAUTHLIB_INSECURE_TRANSPORT'] = '1'


In [3]:
def get_authenticated_service(CLIENT_SECRETS_FILE):
    credentials = None

    # Création du dossier de cache si inexistant
    if not os.path.exists(CACHE_CREDENTIALS_DIR):
        os.makedirs(CACHE_CREDENTIALS_DIR)

    # Chargement des credentials s'ils existent
    if os.path.exists(CACHE_CREDENTIALS_FILE):
        with open(CACHE_CREDENTIALS_FILE, 'rb') as token:
            credentials = pickle.load(token)

    # Rafraîchissement ou obtention de nouveaux credentials
    if not credentials or not credentials.valid:
        """if credentials and credentials.expired and credentials.refresh_token:
            credentials.refresh(Request())
        else:"""
        flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
        credentials = flow.run_local_server(port=0, access_type='offline', prompt='consent')

        # Sauvegarde des nouveaux credentials
        with open(CACHE_CREDENTIALS_FILE, 'wb') as token:
            pickle.dump(credentials, token)

    return build(API_SERVICE_NAME, API_VERSION, credentials=credentials)

# Authentification
service = get_authenticated_service("creds.json")

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=279167112900-c356bf03cvttterb16sbtfa74gnpbote.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A62052%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.readonly&state=6eeAXIrj3ejmYtqIKwbyUplZryslet&access_type=offline&prompt=consent


In [4]:
def search_low_view_music(service, query="music", max_results=50, view_threshold=1000, topic="", coord="28.6139,77.2088", rad="500km", page_token=None):
    request = service.search().list(
        q=query,
        part="snippet",
        type="video",
        maxResults=max_results,
        location=coord,
        locationRadius=rad,
        topicId=topic,
        pageToken=page_token  # Ajout du token de page
    )
    response = request.execute()

    next_page_token = response.get("nextPageToken")  # Récupère le token de la page suivante
    video_ids = [item["id"]["videoId"] for item in response["items"]]

    request = service.videos().list(
        part="snippet,statistics,contentDetails",
        id=",".join(video_ids)
    )
    response = request.execute()

    low_view_videos = []
    strong_view_videos = []
    for video in response["items"]:
        view_count = int(video["statistics"].get("viewCount", 0))
        duration = video["contentDetails"].get("duration", None)
        if duration is None:
            continue  # Ignore les vidéos sans durée

        video_data = {
            "title": video["snippet"]["title"],
            "views": view_count,
            "duration": duration,
            "url": video["id"]
        }

        if view_count < view_threshold:
            low_view_videos.append(video_data)
        else:
            strong_view_videos.append(video_data)

    return low_view_videos, strong_view_videos, next_page_token  # Retourne aussi le token


In [5]:
def format_duration(iso_duration):
    duration = isodate.parse_duration(iso_duration)  # Convertit en objet timedelta
    return duration.seconds


In [6]:
topics = [("hiphop", "/m/0glt670"), ("jazz","/m/03_d0"),("inde","/m/05rwpb"),("electro","/m/02lkt"),("pop","/m/064t9"),("rock","/m/06by7")]
coordonnée= [("france","48.8566,2.3522", "500km"), ("Copenhagen", "55.6761,12.5683"), ("","28.6139,77.2088"), ("UK", "51.5072,0.1276", "200km")]

In [7]:
page_token = None
all_results = []
for i in topics:
    for _ in range(5):  # Faire plusieurs requêtes pour obtenir plus de variété
        m_low, m_strong, page_token = search_low_view_music(service, query="-live|Live|IA|ia|short", view_threshold=20000, topic=i[1], coord=coordonnée[0][1], rad=coordonnée[0][2], page_token=page_token)
        all_results.extend(m_low)
        if not page_token:
            break  # Arrêter si plus de pages disponibles


In [8]:
res = []
for k in all_results:
        if format_duration(k["duration"])>120 and format_duration(k["duration"])<600:
            res.append((k["url"], coordonnée[0][0], i[0]))

In [9]:
db.insert_table(res)

error:  183  duplicate entry or constraint issue


In [10]:
print(len(res))

473
